cluster - pixel axis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools

def calculate_distance(line1, line2):
    # 计算两个线段之间的距离
    x1, ymin1, ymax1 = line1
    x2, ymin2, ymax2 = line2
    return np.sqrt((x1 - x2)**2 + ((ymin1 + ymax1)/2 - (ymin2 + ymax2)/2)**2)

def hierarchical_clustering(lines, distance_threshold):
    clusters = []
    for line in lines:
        added_to_cluster = False
        for cluster in clusters:
            if any(calculate_distance(line, existing_line) <= distance_threshold for existing_line in cluster):
                cluster.append(line)
                added_to_cluster = True
                break
        if not added_to_cluster:
            clusters.append([line])
    
    # 过滤掉只有1-2个线段的团块
    clusters = [cluster for cluster in clusters if len(cluster) > 3]
    
    return clusters

def cluster_and_plot(csv_file, distance_threshold=10):
    # 读取CSV文件
    df = pd.read_csv(csv_file)

    # 筛选class=1的行
    df_class_1 = df[df['class'] == 1]

    # 提取需要的列(x, ymin, ymax)
    lines = df_class_1[['x channel', 'ymin', 'ymax']].values

    # 聚类
    clusters = hierarchical_clustering(lines, distance_threshold)

    # 计算每个团块的中心坐标
    cluster_centers = []
    for cluster in clusters:
        center_x = np.mean([line[0] for line in cluster])
        center_ymin = np.mean([line[1] for line in cluster])
        center_ymax = np.mean([line[2] for line in cluster])
        cluster_centers.append((center_x, center_ymin, center_ymax))

    # 颜色循环器
    colors = itertools.cycle(plt.cm.tab20.colors)

    # 绘制线段和团块中心
    plt.figure(figsize=(10, 8))
    for i, cluster in enumerate(clusters):
        color = next(colors)
        for line in cluster:
            x, ymin, ymax = line
            plt.plot([x, x], [ymin, ymax], '-', color=color)
            # plt.text(x, ymin, f'({x}, {ymin})', fontsize=8)
            # plt.text(x, ymax, f'({x}, {ymax})', fontsize=8)
        
        center_x, center_ymin, center_ymax = cluster_centers[i]
        plt.plot(center_x, (center_ymin + center_ymax) / 2, 'ro')  # 绘制红色点

    plt.xlabel('x channel')
    plt.ylabel('y axis')
    plt.title('Clusters of (x, ymin, ymax)')
    plt.show()

    # 输出团块中心坐标
    for i, center in enumerate(cluster_centers):
        center_x, center_ymin, center_ymax = center
        print(f'Cluster {i}: Center (x, ymin, ymax) = ({center_x}, {center_ymin}, {center_ymax})')

# 使用示例
csv_file = '/Users/naoj306/Desktop/find/aquila_12/location/nonan_diff_1-99_3-3/location-30.csv'
cluster_and_plot(csv_file, distance_threshold=10)  # 使用更宽松的阈值


cluster - hms axis

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

def calculate_distance(line1, line2):
    # 计算两个线段之间的距离
    x1, ymin1, ymax1 = line1
    x2, ymin2, ymax2 = line2
    return np.sqrt((x1 - x2)**2 + ((ymin1 + ymax1)/2 - (ymin2 + ymax2)/2)**2)

def hierarchical_clustering(lines, distance_threshold):
    clusters = []
    for line in lines:
        added_to_cluster = False
        for cluster in clusters:
            if any(calculate_distance(line, existing_line) <= distance_threshold for existing_line in cluster):
                cluster.append(line)
                added_to_cluster = True
                break
        if not added_to_cluster:
            clusters.append([line])
    
    # 过滤掉只有1-2个线段的团块
    clusters = [cluster for cluster in clusters if len(cluster) > 2]
    
    return clusters

def cluster_and_plot(csv_file, fits_file, distance_threshold=10):
    # 读取CSV文件
    df = pd.read_csv(csv_file)

    # 筛选class=1的行
    df_class_1 = df[df['class'] == 1]

    # 提取需要的列(x, ymin, ymax)
    lines = df_class_1[['x channel', 'ymin', 'ymax']].values

    # 聚类
    clusters = hierarchical_clustering(lines, distance_threshold)

    # 计算每个团块的中心坐标
    cluster_centers = []
    for cluster in clusters:
        center_x = np.mean([line[0] for line in cluster])
        center_ymin = np.mean([line[1] for line in cluster])
        center_ymax = np.mean([line[2] for line in cluster])
        cluster_centers.append((center_x, center_ymin, center_ymax))

    # 打开FITS文件并获取WCS
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        wcs = WCS(header).dropaxis(2)  # Drop the velocity dimension

    # 将中心坐标转换为WCS坐标
    cluster_wcs_coords = []
    for center_x, center_ymin, center_ymax in cluster_centers:
        center_y = (center_ymin + center_ymax) / 2
        ra, dec = wcs.all_pix2world(center_x, center_y, 0)
        cluster_wcs_coords.append((ra, dec))

    # 颜色循环器
    colors = itertools.cycle(plt.cm.tab20.colors)

    # 绘制线段和团块中心
    plt.figure(figsize=(10, 8))
    for i, cluster in enumerate(clusters):
        color = next(colors)
        for line in cluster:
            x, ymin, ymax = line
            plt.plot([x, x], [ymin, ymax], '-', color=color)
        
        center_x, center_ymin, center_ymax = cluster_centers[i]
        center_y = (center_ymin + center_ymax) / 2
        ra, dec = cluster_wcs_coords[i]
        coord = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')
        plt.plot(center_x, center_y, 'ro')  # 绘制红色点
        plt.text(center_x, center_y, f'({coord.to_string("hmsdms", precision=2)})', fontsize=8, color='red')

    # 设置坐标轴单位为 hms
    ax = plt.gca()
    ra_axis = np.linspace(0, wcs.pixel_shape[0] - 1, num=5)
    dec_axis = np.linspace(0, wcs.pixel_shape[1] - 1, num=5)
    ra_pix, dec_pix = np.meshgrid(ra_axis, dec_axis)
    ra_deg, dec_deg = wcs.all_pix2world(ra_pix, dec_pix, 0)
    ra_hms = [SkyCoord(ra=ra*u.degree, dec=0*u.degree, frame='icrs').ra.to_string(unit=u.hour, sep=':', precision=2) for ra in ra_deg[0]]
    dec_dms = [SkyCoord(ra=0*u.degree, dec=dec*u.degree, frame='icrs').dec.to_string(unit=u.degree, sep=':', precision=2) for dec in dec_deg[:, 0]]
    ax.set_xticks(ra_axis)
    ax.set_xticklabels(ra_hms)
    ax.set_yticks(dec_axis)
    ax.set_yticklabels(dec_dms)

    plt.xlabel('RA (hms)')
    plt.ylabel('Dec (dms)')
    plt.title('Clusters of (x, ymin, ymax)')
    plt.show()

    # 输出团块中心坐标
    for i, (ra, dec) in enumerate(cluster_wcs_coords):
        coord = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')
        print(f'Cluster {i}: Center (RA, Dec) = {coord.to_string("hmsdms", precision=2)}')
    
# 使用示例
csv_file = '/Users/naoj306/Desktop/find/aquila_12/location/nonan_diff_1-99_3-3/location-30.csv'
fits_file = "/Users/naoj306/Desktop/DB/SFP/AquilaRift_12CO_21.7arcsec_vel0.1_sph_v1.0.fits"
cluster_and_plot(csv_file, fits_file, distance_threshold=10)  # 使用更宽松的阈值


compare with 12CO outflow only (nakamura)

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

def calculate_distance(line1, line2):
    # 计算两个线段之间的距离
    x1, ymin1, ymax1 = line1
    x2, ymin2, ymax2 = line2
    return np.sqrt((x1 - x2)**2 + ((ymin1 + ymax1)/2 - (ymin2 + ymax2)/2)**2)

def hierarchical_clustering(lines, distance_threshold):
    clusters = []
    for line in lines:
        added_to_cluster = False
        for cluster in clusters:
            if any(calculate_distance(line, existing_line) <= distance_threshold for existing_line in cluster):
                cluster.append(line)
                added_to_cluster = True
                break
        if not added_to_cluster:
            clusters.append([line])
    
    # 过滤掉只有1-2个线段的团块
    clusters = [cluster for cluster in clusters if len(cluster) > 2]
    
    return clusters

def cluster_and_plot(csv_file, fits_file, extra_coords, distance_threshold=10):
    # 读取CSV文件
    df = pd.read_csv(csv_file)

    # 筛选class=1的行
    df_class_1 = df[df['class'] == 1]

    # 提取需要的列(x, ymin, ymax)
    lines = df_class_1[['x channel', 'ymin', 'ymax']].values

    # 聚类
    clusters = hierarchical_clustering(lines, distance_threshold)

    # 计算每个团块的中心坐标
    cluster_centers = []
    for cluster in clusters:
        center_x = np.mean([line[0] for line in cluster])
        center_ymin = np.mean([line[1] for line in cluster])
        center_ymax = np.mean([line[2] for line in cluster])
        cluster_centers.append((center_x, center_ymin, center_ymax))

    # 打开FITS文件并获取WCS
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        wcs = WCS(header).dropaxis(2)  # Drop the velocity dimension

    # 将中心坐标转换为WCS坐标
    cluster_wcs_coords = []
    for center_x, center_ymin, center_ymax in cluster_centers:
        center_y = (center_ymin + center_ymax) / 2
        ra, dec = wcs.all_pix2world(center_x, center_y, 0)
        cluster_wcs_coords.append((ra, dec))

    # 颜色循环器
    colors = itertools.cycle(plt.cm.tab20.colors)

    # 绘制线段和团块中心
    plt.figure(figsize=(10, 10))
    for i, cluster in enumerate(clusters):
        color = next(colors)
        for line in cluster:
            x, ymin, ymax = line
            plt.plot([x, x], [ymin, ymax], '-', color=color)
        
        center_x, center_ymin, center_ymax = cluster_centers[i]
        center_y = (center_ymin + center_ymax) / 2
        ra, dec = cluster_wcs_coords[i]
        coord = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')
        plt.plot(center_x, center_y, 'ro')  # 绘制红色点
        # plt.text(center_x, center_y, f'({coord.to_string("hmsdms", precision=2)})', fontsize=8, color='red')

    # 添加额外的RA-Dec数据点
    extra_skycoords = SkyCoord(extra_coords, unit=(u.hourangle, u.deg), frame='icrs')
    extra_pix_coords = wcs.world_to_pixel(extra_skycoords)
    plt.scatter(extra_pix_coords[0], extra_pix_coords[1], edgecolor='green', facecolor='none', marker='s', s=100, label='Extra Data')

    # 设置坐标轴单位为 hms
    ax = plt.gca()
    ra_axis = np.linspace(0, wcs.pixel_shape[0] - 1, num=5)
    dec_axis = np.linspace(0, wcs.pixel_shape[1] - 1, num=5)
    ra_pix, dec_pix = np.meshgrid(ra_axis, dec_axis)
    ra_deg, dec_deg = wcs.all_pix2world(ra_pix, dec_pix, 0)
    ra_hms = [SkyCoord(ra=ra*u.degree, dec=0*u.degree, frame='icrs').ra.to_string(unit=u.hour, sep=':', precision=2) for ra in ra_deg[0]]
    dec_dms = [SkyCoord(ra=0*u.degree, dec=dec*u.degree, frame='icrs').dec.to_string(unit=u.degree, sep=':', precision=2) for dec in dec_deg[:, 0]]
    ax.set_xticks(ra_axis)
    ax.set_xticklabels(ra_hms)
    ax.set_yticks(dec_axis)
    ax.set_yticklabels(dec_dms)

    plt.xlabel('RA (hms)')
    plt.ylabel('Dec (dms)')
    plt.title('Clusters of (x, ymin, ymax)')
    plt.legend()
    plt.show()

    # 输出团块中心坐标
    for i, (ra, dec) in enumerate(cluster_wcs_coords):
        coord = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')
        print(f'Cluster {i}: Center (RA, Dec) = {coord.to_string("hmsdms", precision=2)}')
    
# 使用示例
csv_file = '/Users/naoj306/Desktop/find/aquila_12/location/nonan_diff_1-99_3-3/location-30.csv'
fits_file = "/Users/naoj306/Desktop/DB/SFP/AquilaRift_12CO_21.7arcsec_vel0.1_sph_v1.0.fits"

extra_coords = [
    "18:29:03.62 -01:39:03.0",
    "18:29:05.52 -01:41:53.6", "18:29:12.68 -01:46:18.4",
    "18:29:23.69 -01:38:54.0",
    "18:29:59.67 -02:00:58.7", "18:30:01.50 -02:10:25.5", "18:30:03.68 -01:36:29.4",
    "18:30:04.19 -02:03:05.5", "18:30:12.44 -02:06:53.6", "18:30:14.93 -01:33:34.9",
    "18:30:17.64 -02:09:59.3", "18:30:28.98 -01:56:03.2", "18:30:29.28 -01:56:50.6"
]
cluster_and_plot(csv_file, fits_file, extra_coords, distance_threshold=10)  # 使用更宽松的阈值


compare with 12CO all, MHO, SED

12CO https://academic.oup.com/pasj/article/71/Supplement_1/S3/5564843
MHO https://iopscience.iop.org/article/10.1088/0067-0049/219/2/21
SED https://www.aanda.org/articles/aa/full_html/2011/11/aa17132-11/aa17132-11.html

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

def calculate_distance(line1, line2):
    # 计算两个线段之间的距离
    x1, ymin1, ymax1 = line1
    x2, ymin2, ymax2 = line2
    return np.sqrt((x1 - x2)**2 + ((ymin1 + ymax1)/2 - (ymin2 + ymax2)/2)**2)

def hierarchical_clustering(lines, distance_threshold):
    clusters = []
    for line in lines:
        added_to_cluster = False
        for cluster in clusters:
            if any(calculate_distance(line, existing_line) <= distance_threshold for existing_line in cluster):
                cluster.append(line)
                added_to_cluster = True
                break
        if not added_to_cluster:
            clusters.append([line])
    
    # 过滤掉只有1-2个线段的团块
    clusters = [cluster for cluster in clusters if len(cluster) > 2]
    
    return clusters

def cluster_and_plot(csv_file, fits_file, extra_coords1, extra_coords2, distance_threshold):
    # 读取CSV文件
    df = pd.read_csv(csv_file)

    # 筛选class=1的行
    df_class_1 = df[df['class'] == 1]

    # 提取需要的列(x, ymin, ymax)
    lines = df_class_1[['x channel', 'ymin', 'ymax']].values

    # 聚类
    clusters = hierarchical_clustering(lines, distance_threshold)

    # 计算每个团块的中心坐标
    cluster_centers = []
    for cluster in clusters:
        center_x = np.mean([line[0] for line in cluster])
        center_ymin = np.mean([line[1] for line in cluster])
        center_ymax = np.mean([line[2] for line in cluster])
        cluster_centers.append((center_x, center_ymin, center_ymax))

    # 打开FITS文件并获取WCS
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        wcs = WCS(header).dropaxis(2)  # Drop the velocity dimension

    # 将中心坐标转换为WCS坐标
    cluster_wcs_coords = []
    for center_x, center_ymin, center_ymax in cluster_centers:
        center_y = (center_ymin + center_ymax) / 2
        ra, dec = wcs.all_pix2world(center_x, center_y, 0)
        cluster_wcs_coords.append((ra, dec))

    # 颜色循环器
    colors = itertools.cycle(plt.cm.tab20.colors)

    # 绘制线段和团块中心
    plt.figure(figsize=(9, 8))
    for i, cluster in enumerate(clusters):
        color = next(colors)
        for line in cluster:
            x, ymin, ymax = line
            plt.plot([x, x], [ymin, ymax], '-', color=color)
        
        center_x, center_ymin, center_ymax = cluster_centers[i]
        center_y = (center_ymin + center_ymax) / 2
        ra, dec = cluster_wcs_coords[i]
        coord = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')
        plt.plot(center_x, center_y, 'ro')  # 绘制红色点
        # plt.text(center_x, center_y, f'({coord.to_string("hmsdms", precision=2)})', fontsize=8, color='red')

    # 添加额外的RA-Dec数据点
    extra_skycoords1 = SkyCoord(extra_coords1, unit=(u.hourangle, u.deg), frame='icrs')
    extra_pix_coords1 = wcs.world_to_pixel(extra_skycoords1)
    
    extra_skycoords2 = SkyCoord(extra_coords2, unit=(u.hourangle, u.deg), frame='icrs')
    extra_pix_coords2 = wcs.world_to_pixel(extra_skycoords2)
    
    extra_skycoords3 = SkyCoord(extra_coords3, unit=(u.hourangle, u.deg), frame='icrs')
    extra_pix_coords3 = wcs.world_to_pixel(extra_skycoords3)
    
    # 绘制线段图层上面的标注
    plt.scatter(extra_pix_coords1[0], extra_pix_coords1[1], edgecolor='green', facecolor='none', s=100, label='12CO-NRO-2mm', zorder=3)
    plt.scatter(extra_pix_coords2[0], extra_pix_coords2[1], edgecolor='blue', facecolor='none', marker='^', s=100, label='MHO-Spitzer-2.12um', zorder=3)
    plt.scatter(extra_pix_coords3[0], extra_pix_coords3[1], edgecolor='magenta', facecolor='none', marker="d", s=50, label='SED-MANBO-1.2mm', zorder=3)

    # 设置坐标轴单位为 hms
    ax = plt.gca()
    ra_axis = np.linspace(0, wcs.pixel_shape[0] - 1, num=5)
    dec_axis = np.linspace(0, wcs.pixel_shape[1] - 1, num=5)
    ra_pix, dec_pix = np.meshgrid(ra_axis, dec_axis)
    ra_deg, dec_deg = wcs.all_pix2world(ra_pix, dec_pix, 0)
    ra_hms = [SkyCoord(ra=ra*u.degree, dec=0*u.degree, frame='icrs').ra.to_string(unit=u.hour, sep=':', precision=2) for ra in ra_deg[0]]
    dec_dms = [SkyCoord(ra=0*u.degree, dec=dec*u.degree, frame='icrs').dec.to_string(unit=u.degree, sep=':', precision=2) for dec in dec_deg[:, 0]]
    ax.set_xticks(ra_axis)
    ax.set_xticklabels(ra_hms)
    ax.set_yticks(dec_axis)
    ax.set_yticklabels(dec_dms)

    plt.xlabel('RA (hms)')
    plt.ylabel('Dec (dms)')
    plt.title('Clusters of (x, ymin, ymax)')
    plt.legend()
    plt.show()

    # 输出团块中心坐标
    for i, (ra, dec) in enumerate(cluster_wcs_coords):
        coord = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')
        print(f'Cluster {i}: Center (RA, Dec) = {coord.to_string("hmsdms", precision=2)}')
    
# 使用示例
csv_file = '/Users/naoj306/Desktop/find/aquila_12/location/nonan_diff_1-99_3-3/location-30.csv'
fits_file = "/Users/naoj306/Desktop/DB/SFP/AquilaRift_12CO_21.7arcsec_vel0.1_sph_v1.0.fits"

with open("/Users/naoj306/Desktop/find/candidate/12CO-NRO2mm_10.1093:pasj:psz057.txt", "r") as file: 
    extra_coords1 = file.read().splitlines()
with open("/Users/naoj306/Desktop/find/candidate/MHO-Spitzer2.12um_10.1088:0067-0049:219:2:21 .txt", "r") as file: 
    extra_coords2 = file.read().splitlines()
with open("/Users/naoj306/Desktop/find/candidate/SED-MANBO1.2mm_10.1051:0004-6361:201117132.txt", "r") as file: 
    extra_coords3 = file.read().splitlines()    
    
cluster_and_plot(csv_file, fits_file, extra_coords1, extra_coords2, distance_threshold=15)  # 使用更宽松的阈值
